<a href="https://colab.research.google.com/github/Pi-vardhan/Pi-vardhan/blob/Project/Water%20Tank%20Drain%20Time%20Prediction_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mounting Drive**

In [170]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Libraries

In [171]:
import numpy 
import math 
import random
import pandas 

**Data Input**

In [172]:
tank = pandas.read_csv('/content/drive/MyDrive/Tank_post.csv')
tank.head(7)

,DOW,Fllled,Drained
0,Sunday,18,8
1,Monday,10,11
2,Tuesday,13,7
3,Wednesday,8,16
4,Thursday,18,16
5,Friday,18,22
6,Saturday,24,14


**Calculate Difference** [ Time Taken to Drain = Drain Time - Filled Time ]

In [173]:
ftime=tank['Fllled']
dtime=tank['Drained']
timetook=dtime-ftime
tank = pandas.concat([tank, timetook],axis = 1)
#tank.drop(['Fllled'],axis=1,inplace = True)
tank.drop(['Drained'],axis=1,inplace = True)
tank = tank.rename(columns=({0:'Drain Time'}))
#tank.head(7)

No Negative Numbers

In [174]:
data = tank['Drain Time']
i=0
for j in data:
  if j<1:
    tank.loc[i,'Drain Time'] = (j*-1)+24
  i=i+1
tank.head(7)

,DOW,Fllled,Drain Time
0,Sunday,18,34
1,Monday,10,1
2,Tuesday,13,30
3,Wednesday,8,8
4,Thursday,18,26
5,Friday,18,4
6,Saturday,24,34


Outliers Detection

In [175]:
def detect_outliers_zscore(data):
    outliers = []
    thres = 2
    mean = numpy.mean(data)
    std = numpy.std(data)
    for i in data:
        z_score = (i-mean)/std
        if (numpy.abs(z_score) > thres):
            outliers.append(i)
    return outliers

In [176]:
sample_outliers_Drain_Time = detect_outliers_zscore(tank['Drain Time'])
#sample_outliers_Fllled = detect_outliers_zscore(tank['Fllled'])
#print('No.of Outliers in "Fllled" Column:', len(sample_outliers_Fllled))
#print("Outliers in 'Fllled' column : ", sample_outliers_Fllled)
print('No.of Outliers in "Drain Time" Column:', len(sample_outliers_Drain_Time))
print("Outliers in 'Drain Time' column : ", sample_outliers_Drain_Time)

No.of Outliers in "Drain Time" Column: 3
Outliers in 'Drain Time' column :  [63, 75, 106]


In [177]:
from scipy.stats import zscore
z = numpy.abs(zscore(tank['Drain Time']))
tank['Zscore_Drain_Time'] = z
outliers_Drain_Time=len(tank[tank['Zscore_Drain_Time']>1])
data_z = tank[tank['Zscore_Drain_Time']<1.4]
tank=data_z.drop(['Zscore_Drain_Time'], axis=1)

In [178]:
print(tank.describe())

           Fllled  Drain Time
count  270.000000  270.000000
mean    13.977778   22.685185
std      6.940498   12.144153
min      1.000000    3.000000
25%      8.000000   10.000000
50%     15.000000   26.000000
75%     20.000000   32.000000
max     26.000000   43.000000


Missing Value Treatment

In [179]:
print(tank.isnull().sum())

DOW           0
Fllled        0
Drain Time    0
dtype: int64



Catergorical CONVERSION

In [180]:
status = pandas.get_dummies(tank['DOW'], drop_first = True)
tank = pandas.concat([tank, status],axis = 1)
tank.drop(['DOW'],axis = 1, inplace = True)
tank

,Fllled,Drain Time,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,18,34,0,0,1,0,0,0
2,13,30,0,0,0,0,1,0
3,8,8,0,0,0,0,0,1
4,18,26,0,0,0,1,0,0
5,18,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
296,8,5,1,0,0,0,0,0
299,11,6,0,0,0,1,0,0
300,18,29,0,0,0,0,0,0
301,14,36,0,1,0,0,0,0


DATA Split

In [181]:
from sklearn.model_selection import train_test_split
tank_train, tank_test= train_test_split(tank,train_size = 0.70, test_size = 0.30, random_state = 1438)

DATA Normalization

In [182]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
nv = ['Drain Time','Fllled']
tank_train[nv] = scaler.fit_transform(tank_train[nv])
tank_test[nv]= scaler.fit_transform(tank_test[nv])

tank_train

,Fllled,Drain Time,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
86,0.24,0.700,0,0,1,0,0,0
203,0.36,0.125,0,1,0,0,0,0
14,0.64,0.900,0,0,1,0,0,0
267,0.96,0.700,0,0,1,0,0,0
149,0.56,0.050,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
195,0.24,0.100,0,0,0,0,0,0
234,0.16,0.550,0,0,0,0,1,0
240,0.92,0.625,1,0,0,0,0,0
178,0.16,0.400,0,0,0,0,1,0


INPUT - OUTPUT Data Split

In [183]:
x_train = tank_train
x_test = tank_test
y_train = tank_train.pop('Drain Time')
y_test = tank_test.pop('Drain Time')

**Model Training**

In [184]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression()

In [185]:
m = regressor.coef_
c = regressor.intercept_,
print("Estimated model weights, m:" , m)
print("Estimated model intercept, c:" , c)

Estimated model weights, m: [ 0.65991238  0.03696386  0.01996042 -0.04029163  0.01871586  0.07672667
  0.03251033]
Estimated model intercept, c: (0.12557721791254467,)


Prediction on Training & Testing Data

In [186]:
y_pred_test =  regressor.predict(x_test)
y_pred_train =  regressor.predict(x_train)

Error Metric Computation

In [187]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error :', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error      :', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error    :', mae)

Root mean square error : 0.23589107433036702
Mean square error      : 0.05564459894873474
Mean absolute error    : 0.19576648494132415


In [188]:
#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error :', rmse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error      :', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error    :', mae)
     

Root mean square error : 0.2783595724882302
Mean square error      : 0.07748405159583029
Mean absolute error    : 0.22478428807966913


SAVE Model

In [189]:
import pickle
# Save the model
filename = 'model.pkl'
pickle.dump(regressor, open(filename, 'wb'))

Deployment

In [190]:
list_of_columns = tank.columns
input_data = pandas.DataFrame(columns=list_of_columns)
input_data.drop(['Drain Time'],axis='columns',inplace=True)
m=t=w=th=f=s=su=0
day = str(input('Day:'))
if day == 'Monday':
  m=1
if day == 'Tuesday':
  t=1
if day == 'Wednesday':
  w=1
if day == 'Thursday':
  th=1
if day == 'Saturday':
  s=1
if day == 'Sunday':
  su=1
input_data.at[0,'Monday'] = m
input_data.at[0,'Tuesday'] = t
input_data.at[0,'Wednesday'] = w
input_data.at[0,'Thursday'] = th
input_data.at[0,'Saturday'] = s
input_data.at[0,'Sunday'] = su

input_data.at[0,'Fllled'] = time = int(input('Enter Time: '))
while time>23 or time<0:
  print('Invalid Time!!')
  input_data.at[0,'Fllled'] = time = int(input('Enter Time: '))

input_data['Fllled']=(input_data['Fllled']-tank['Fllled'].min())/(tank['Fllled'].max()-tank['Fllled'].min())
input_data['Monday']=(input_data['Monday']-tank['Monday'].min())/(tank['Monday'].max()-tank['Monday'].min())
input_data['Tuesday']=(input_data['Tuesday']-tank['Tuesday'].min())/(tank['Tuesday'].max()-tank['Tuesday'].min())
input_data['Wednesday']=(input_data['Wednesday']-tank['Wednesday'].min())/(tank['Wednesday'].max()-tank['Wednesday'].min())
input_data['Thursday']=(input_data['Thursday']-tank['Thursday'].min())/(tank['Thursday'].max()-tank['Thursday'].min())
input_data['Saturday']=(input_data['Saturday']-tank['Saturday'].min())/(tank['Saturday'].max()-tank['Saturday'].min())
input_data['Sunday']=(input_data['Sunday']-tank['Sunday'].min())/(tank['Sunday'].max()-tank['Sunday'].min())





Day:Sunday
Enter Time: 9


Defining Basic Codes

In [191]:
def ampm(x):
  if x<12 :
    return 'AM'
  else:
    return 'PM'

def t12(x):
  if x>12:
    return x-12
  else:
    return x

def frint(x):
  if x>47:
    x = x-48
    print('Water Tank will be Drained by', t12(x),ampm(x),'Day After Tommorow.')
  elif x>23:
    x = x-24
    print('Water Tank Will Be Emptied by', t12(x),ampm(x), 'Tommorow.') 
  else:
    print('Water Tank Will Be Emptied by',t12(x),ampm(x),'Today.')

# ***Final Outputs***

In [192]:
y_pred =  regressor.predict(input_data)
Dt = y_pred*(tank['Drain Time'].max()-tank['Drain Time'].min())+tank['Drain Time'].min()
print('Predicted Drain Time =',int(Dt),'hrs')
et = time+int(Dt)
frint(et)

Predicted Drain Time = 14 hrs
Water Tank Will Be Emptied by 11 PM Today.
